# Sparse Embeddings

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
from datasets import load_dataset
from qdrant_client import models, QdrantClient
from fastembed import SparseTextEmbedding

from src import load_data, embeddings


# Params

In [ ]:
INDEX = "ragquas-sparse-01"
RETRIEVE_K=5

# Environment variables

In [ ]:
load_dotenv()


# Clients

In [ ]:
client_qdrant = QdrantClient(":memory:") 


# Load Data

In [ ]:
df_ragquas = load_data.ragquas()
df_ragquas.head(2)

In [ ]:
df_docs = load_data.prepare_ragquas(df_ragquas)
df_docs.info()

# How Sparse Vectors Works 

In [10]:
documents = [
    "You should stay, study and sprint.",
    "History can only prepare us to be surprised yet again.",
]

encoder = SparseTextEmbedding(model_name="Qdrant/bm25")
sp_embeddings = list(encoder.embed(documents))


print(f"{sp_embeddings[0].values=}")
print(f"{sp_embeddings[0].indices=}")

sp_embeddings[0].values=array([1.67868852, 1.67868852, 1.67868852])
sp_embeddings[0].indices=array([1881538586,  150760872, 1932363795])


In [12]:
type(sp_embeddings[0])

fastembed.sparse.sparse_embedding_base.SparseEmbedding

# Index

In [13]:
lst_embeddings = []

corpus_encoded = list(encoder.embed(df_docs['text_']))

for idx in range(len(corpus_encoded)):
    sp_vec = corpus_encoded[idx]
    lst_embeddings.append({
        "sparse": models.SparseVector(
            indices=sp_vec.indices,
            values=sp_vec.values
            )
        }
    )

print(f"{len(lst_embeddings)=}")

len(lst_embeddings)=183


In [14]:
lst_pts = embeddings.create_index_points(
    lst_embeddings,
    df_docs,
)
print(f"{len(lst_pts)=}")

len(lst_pts)=183


In [15]:
lst_qdrant_pts = embeddings.convert_to_qdrant_points(lst_pts)
print(f"{len(lst_qdrant_pts)=}")

len(lst_qdrant_pts)=183


In [16]:
client_qdrant.create_collection(
    collection_name=INDEX,
    vectors_config={},
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(),
        }
)

client_qdrant.upsert(
    collection_name=INDEX,
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [17]:
_q_text = df_docs.iloc[0:1]['question']

list(encoder.embed(documents))[0]


SparseEmbedding(values=array([1.67868852, 1.67868852, 1.67868852]), indices=array([1881538586,  150760872, 1932363795]))

In [18]:
_q_text = df_docs.iloc[0:1]['question']

_q_encoded = next(encoder.embed(_q_text))



_q_vec = models.SparseVector(
    indices=_q_encoded.indices,
    values=_q_encoded.values
    )

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        INDEX,
        _q_vec,
        retrieve_k=3,
        using='sparse'  # passed by *args, **kwargs to query_points()
)



Question: 0    ¿Cuál es la forma más fácil de reclamar cuando...
Name: question, dtype: object
point.id='83fe5701-ef07-42e0-86cb-db59f61bafa5'
point.score=31.911108016967773
Doc: Conoce las compensaciones cuando se retrasa tu vuelo

Llegas al aeropuerto y tu vuelo va con retraso, ¿conoces cuáles son tus derechos y si puedes pedir reclamar una compensación? Desde AirHelp te ayudamos con los trámites para que tu viaje no se convierta en una pesadilla y puedas conocer todos los casos en los que puedes hacer una reclamación y el plazo para realizar los reclamos. ¡Te ayudaremos a reclamar tu indemnización por si tu vuelo se retrasa!

Desplazarse en avión es para muchas person...
idx: 0, j: 2
------------------------------
point.id='a25e57b8-a961-47ab-bc6c-fc20067ef63a'
point.score=31.027894973754883
Doc: 5 páginas donde poder reclamar tus vuelos por retraso o cancelación de manera gratuita
Irina Pérez

    En muchas ocasiones no se realiza la reclamación de un vuelo por retraso o cancelació